# CrewAI Workshop

In [5]:
!pip install --upgrade pip

# Install the mains crewAI package
!pip install crewai==0.30.11

# Install the main crewAI package and the tools package
# that includes a series of helpful tools for your agents
!pip install 'crewai[tools]'==0.2.6

In [2]:
!pip install langchain==0.1.17
!pip install boto3==1.34.95
!pip install langchain-aws==0.1.4 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 37.9 MB/s eta 0:00:00
  Attempting uninstall: langchain
    Found existing installation: langchain 0.1.20
    Uninstalling langchain-0.1.20:
      Successfully uninstalled langchain-0.1.20
  Using cached jmespath-1.0.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached s3transfer-0.10.1-py3-none-any.whl.metadata (1.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 136.6 MB/s eta 0:00:0000:010:01
Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
Using cached s3transfer-0.10.1-py3-none-any.whl (82 kB)


## Step 1: init LLM

In [4]:
from langchain_aws import ChatBedrock

def init_bedrock_chat(model_id='anthropic.claude-3-sonnet-20240229-v1:0', region_name='us-west-2'):

    model_kwargs =  { 
        "max_tokens": 4096,
        "temperature": 0.0
    }
    bedrock_chat = ChatBedrock(model_id=model_id, model_kwargs=model_kwargs, region_name=region_name)
    print("boto3 Bedrock client successfully created!")
    return bedrock_chat

In [5]:
bedrock_llm = init_bedrock_chat()
bedrock_llm.invoke('hello')

boto3 Bedrock client successfully created!


AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'usage': {'prompt_tokens': 8, 'completion_tokens': 12, 'total_tokens': 20}}, response_metadata={'model_id': 'anthropic.claude-3-sonnet-20240229-v1:0', 'usage': {'prompt_tokens': 8, 'completion_tokens': 12, 'total_tokens': 20}}, id='run-5185c394-4729-487d-82cc-fa86a80e73da-0')

## Step 2: Create tools

In [6]:
import os
os.environ["SERPER_API_KEY"] = "XXXXXXX"

from crewai_tools import SerperDevTool
search_tool = SerperDevTool()

2024-05-20 23:15:38,979 [embedchain] [INFO] Swapped std-lib sqlite3 with pysqlite3 for ChromaDb compatibility. Your original version was 3.7.17.


## Step 3 Create Agent

In [8]:
from crewai import Agent
# Define your agents with roles and goals
researcher = Agent(
  role='Senior Research Analyst',
  goal='Uncover cutting-edge developments in AI and data science',
  backstory="""You work at a leading tech think tank.
  Your expertise lies in identifying emerging trends.
  You have a knack for dissecting complex data and presenting actionable insights.""",
  verbose=True,
  allow_delegation=False,
  tools=[search_tool],
  llm=bedrock_llm
)
writer = Agent(
  role='Tech Content Strategist',
  goal='Craft compelling content on tech advancements',
  backstory="""You are a renowned Content Strategist, known for your insightful and engaging articles.
  You transform complex concepts into compelling narratives.""",
  verbose=True,
  allow_delegation=True,
  llm=bedrock_llm
)

## Step 4 Create Task

In [10]:
from crewai import Task

task1 = Task(
  description="""Conduct a comprehensive analysis of the latest advancements in AI in 2024.
  Identify key trends, breakthrough technologies, and potential industry impacts.""",
  expected_output="Full analysis report in bullet points",
  agent=researcher
)

task2 = Task(
  description="""Using the insights provided, develop an engaging blog
  post that highlights the most significant AI advancements.
  Your post should be informative yet accessible, catering to a tech-savvy audience.
  Make it sound cool, avoid complex words so it doesn't sound like AI.""",
  expected_output="Full blog post of at least 4 paragraphs",
  agent=writer
)

## Step 5 Kick Off

In [11]:
from crewai import Crew
# Instantiate your crew with a sequential process
crew = Crew(
  agents=[researcher, writer],
  tasks=[task1, task2],
  verbose=2, # You can set it to 1 or 2 to different logging levels
)

# Get your crew to work!
result = crew.kickoff()

print("######################")
print(result)

 [DEBUG]: == Working Agent: Senior Research Analyst
 [INFO]: == Starting Task: Conduct a comprehensive analysis of the latest advancements in AI in 2024.
  Identify key trends, breakthrough technologies, and potential industry impacts.


> Entering new CrewAgentExecutor chain...
Thought: To conduct a comprehensive analysis of the latest advancements in AI in 2024, I will need to search for relevant information on the internet. I should start with a broad search to get an overview of the key developments and trends, and then narrow down to specific areas or technologies as needed.

Action: Search the internet
Action Input: {"search_query": "AI advancements 2024"}
 


Search results: Title: The most important AI trends in 2024 - IBM Blog
Link: https://www.ibm.com/blog/artificial-intelligence-trends/
Snippet: Reality check: more realistic expectations · Multimodal AI · Small(er) language models and open source advancements · GPU shortages and cloud costs ...
---
Title: AI Index Report 202

# App Store Review

In [13]:
!pip install google_play_scraper

  Using cached google_play_scraper-1.2.6-py3-none-any.whl.metadata (50 kB)
Using cached google_play_scraper-1.2.6-py3-none-any.whl (28 kB)


In [7]:

from google_play_scraper import search, Sort, reviews

In [8]:
from langchain.agents import tool 
@tool
def get_google_app_id(app_name:str):
    """Tool to found Google App Store AppID by app name"""
    try:
        result = search(
            app_name,
            lang="en",  # defaults to 'en'
            country="us",  # defaults to 'us'
            n_hits=1  # defaults to 30 (= Google's maximum)
        )
        if len(result)>0:
            return result[0]['appId']
        else:
            return None

    except Exception as e:
        print(e)
        return None

In [9]:
get_google_app_id('starrail')

/data/src/crew-ai-workshop/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


'com.HoYoverse.hkrpgoversea'

In [7]:

import json
from datetime import datetime
from langchain.agents import tool
from google_play_scraper import Sort, reviews


class DateTimeEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, datetime):
            return o.isoformat()
        return super().default(o)


def save_review(app_id, data):
    with open(f"output/{app_id}.json", "w", encoding="utf-8") as file:
        json.dump(data, file, cls=DateTimeEncoder,
                  ensure_ascii=False, indent=4)


@tool
def get_google_play_app_review(app_id: str, country: str = 'us', rank: int = -1, page: int = 1):
    """Tool to found Google App Store App reviews by app id ,country ,rank and page"""
    try:

        filter_score = None
        if rank > 0 and rank < 6:
            filter_score = rank
        print(f"{country=}")
        result, continuation_token = reviews(
            app_id,
            lang='en',  # defaults to 'en'
            country="us" if country == "" else country,  # defaults to 'us'
            sort=Sort.NEWEST,  # defaults to Sort.NEWEST
            count=100,  # defaults to 100
            filter_score_with=filter_score
        )

        if continuation_token:
            print(f'Have continuation token: {continuation_token}')

        save_review(app_id, result)

        return [{"username": review["userName"], "content": review["content"], "score": review["score"]} for review in result]
    except Exception as e:
        print(e)
        return []

In [10]:
get_google_play_app_review('com.miHoYo.GenshinImpact')

country='us'
Have continuation token: <google_play_scraper.features.reviews._ContinuationToken object at 0x7f7c942c5480>


[{'username': 'jhund5', 'content': 'Me likeky', 'score': 2},
 {'username': 'Chet Dy',
  'content': 'My bro did the 1 starrrr😭😭😭😭😭',
  'score': 5},
 {'username': 'Kar Esmane',
  'content': 'This game taught me to be grateful and satisfied with everything I currently have; be it in my life or in Genshin Impact. I really like its casual gameplay because the aesthetics of everything looks really really good, the music and sound designs are phenomenal. Not only that, but the game is also free.',
  'score': 5},
 {'username': 'John Paolo Domingo',
  'content': 'Its better if hoyoverse add switch stat on other artifacts',
  'score': 5},
 {'username': 'NightRose', 'content': 'I love this game', 'score': 5},
 {'username': 'Yvonne Gbesiagbe',
  'content': "Loved this until now. When I got to the quest of Aether meeting Lumine, during the cutscene of Aether fighting the Abyss Herald, the game would freeze and then exit me out. I've tried everything, including uninstalling and reinstalling the game

In [17]:
from crewai import Agent
review_loader = Agent(
    role='App Review Research Analyst',
    goal='If user input is app id, you load App review  from a app id',
    backstory='Specializes in handling and interpreting app reviews documents',
    verbose=True,
    tools=[get_google_play_app_review],
    allow_delegation=False,
    llm=bedrock_llm,
    memory=True,
)

researcher = Agent(
  role='Expert App Store reviews summarizer',
  goal='Found prices issue , network issue, game content issue in every user\'s reviews content, you need keep username',
  backstory="""You are a renowned expert summarizer of AI related 
  Google App Store User reviews, known for your insightful 
  and engaging articles. You transform complex concepts into 
  compelling narratives.""",
  verbose=True,
  allow_delegation=True,
  llm=bedrock_llm
  
)

In [20]:
import sys
from crewai import Task, Crew

def init_app_crew(store:str,app_name:str,country:str, rank:int=-1,page:int=1, file:str="",output_stream=None):
    """
    This function is the main entry point for the application.
    It takes an app name and a review rank , page as input, and returns the result of processing the app reviews.
    """
    
    if output_stream is not None:
            sys.stdout = output_stream
    

    # Get the app ID from the app name using the get_google_app_id tool
    app_id = get_google_app_id(app_name)
        
    # Create a task for the review_reader agent to fetch reviews for the given app ID and rank
    review_loading_task = Task(
        description=f"""Use store: {store} ,app_name:{app_name} app_id: {app_id} and country:{country}, rank: {rank}, page: {page}""",
        agent=review_loader,
        expected_output='A refined finalized version of the blog post in markdown format'
    )

    content_creation_task = Task(
        description="""Using the insights provided by the research Analyst agent, if have different score , please calculate every score the percentage
        develop an engaging  content, you need use markdown format and translate to chinese. """,
        agent=researcher,
        expected_output=f'A refined finalized version of the blog post in markdown format,need include store: {store} , app_id: {app_id} and country:{country},translate to chinese.'
    )

    # Instantiate a Crew with the review_reader and researcher agents, and the tasks to be performed
    # Set the logging level to 2 (more verbose)
    crew = Crew(
        agents=[review_loader, researcher],
        tasks=[review_loading_task, content_creation_task],
        verbose=2, 
    )

    return app_id,crew

In [22]:
app_id, crew=init_app_crew('Google Play', 'genshin', "us", -1)

2024-05-20 23:18:40,172 - 140332172193856 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


In [38]:
result = crew.kickoff()

 [DEBUG]: == Working Agent: App Review Research Analyst
 [INFO]: == Starting Task: Use store: Google Play ,app_name:genshin app_id: com.miHoYo.GenshinImpact and country:us, rank: -1, page: 1


> Entering new CrewAgentExecutor chain...
Thought: To get the app reviews for the Genshin Impact app from the Google Play Store in the US, I will need to use the get_google_play_app_review tool with the provided app ID, country, rank, and page number.

Action: get_google_play_app_review
Action Input: {"app_id": "com.miHoYo.GenshinImpact", "country": "us", "rank": -1, "page": 1}
country='us'
Have continuation token: <google_play_scraper.features.reviews._ContinuationToken object at 0x7fa150dbc6a0>
 

[{'username': 'ilyani_chs', 'content': "3/5 because ain't no way you having me take up this much storage for one update 😭😭??", 'score': 3}, {'username': 'Anis Syahirah', 'content': 'Burnout. Repeated tasks. Play sometimes. Good for beginner though. Bye genshin it has been nice 3 years. 🦅', 'score': 1}

## 将代码整合到一起

In [1]:
from langchain_aws import ChatBedrock
from langchain.agents import tool 
from google_play_scraper import search, Sort, reviews
import json
from datetime import datetime
from langchain.agents import tool
from google_play_scraper import Sort, reviews
from crewai import Agent
import sys
from crewai import Task, Crew

def init_bedrock_chat(model_id='anthropic.claude-3-sonnet-20240229-v1:0', region_name='us-west-2'):

    model_kwargs =  { 
        "max_tokens": 4096,
        "temperature": 0.0
    }
    bedrock_chat = ChatBedrock(model_id=model_id, model_kwargs=model_kwargs, region_name=region_name)
    print("boto3 Bedrock client successfully created!")
    return bedrock_chat


bedrock_llm = init_bedrock_chat()


@tool
def get_google_app_id(app_name:str):
    """Tool to found Google App Store AppID by app name"""
    try:
        result = search(
            app_name,
            lang="en",  # defaults to 'en'
            country="us",  # defaults to 'us'
            n_hits=1  # defaults to 30 (= Google's maximum)
        )
        if len(result)>0:
            return result[0]['appId']
        else:
            return None

    except Exception as e:
        print(e)
        return None



class DateTimeEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, datetime):
            return o.isoformat()
        return super().default(o)


def save_review(app_id, data):
    with open(f"output/{app_id}.json", "w", encoding="utf-8") as file:
        json.dump(data, file, cls=DateTimeEncoder,
                  ensure_ascii=False, indent=4)


@tool
def get_google_play_app_review(app_id: str, country: str = 'us', rank: int = -1, page: int = 1):
    """Tool to found Google App Store App reviews by app id ,country ,rank and page"""
    try:

        filter_score = None
        if rank > 0 and rank < 6:
            filter_score = rank
        print(f"{country=}")
        result, continuation_token = reviews(
            app_id,
            lang='en',  # defaults to 'en'
            country="us" if country == "" else country,  # defaults to 'us'
            sort=Sort.NEWEST,  # defaults to Sort.NEWEST
            count=100,  # defaults to 100
            filter_score_with=filter_score
        )

        if continuation_token:
            print(f'Have continuation token: {continuation_token}')

        # if len(result) > 0:
        #     process_app_review_with_llm(result)
        save_review(app_id, result)
        # with open(f"output/{app_id}.json", "w", encoding="utf-8") as file:
        #     json.dump(result, file, cls=DateTimeEncoder,ensure_ascii=False, indent=4)

        return [{"username": review["userName"], "content": review["content"], "score": review["score"]} for review in result]
    except Exception as e:
        print(e)
        return []
    

review_loader = Agent(
    role='App Review Research Analyst',
    goal='If user input is app id, you load App review  from a app id',
    backstory='Specializes in handling and interpreting app reviews documents',
    verbose=True,
    tools=[get_google_play_app_review],
    allow_delegation=False,
    llm=bedrock_llm,
    memory=True,
)

researcher = Agent(
  role='Expert App Store reviews summarizer',
  goal='Found prices issue , network issue, game content issue in every user\'s reviews content, you need keep username',
  backstory="""You are a renowned expert summarizer of AI related 
  Google App Store User reviews, known for your insightful 
  and engaging articles. You transform complex concepts into 
  compelling narratives.""",
  verbose=True,
  allow_delegation=True,
  llm=bedrock_llm
  
)



def init_app_crew(store:str,app_name:str,country:str, rank:int=-1,page:int=1, file:str="",output_stream=None):
    """
    This function is the main entry point for the application.
    It takes an app name and a review rank , page as input, and returns the result of processing the app reviews.
    """
    
    if output_stream is not None:
            sys.stdout = output_stream
    

    # Get the app ID from the app name using the get_google_app_id tool
    app_id = get_google_app_id(app_name)
        
    # Create a task for the review_reader agent to fetch reviews for the given app ID and rank
    review_loading_task = Task(
        description=f"""Use store: {store} ,app_name:{app_name} app_id: {app_id} and country:{country}, rank: {rank}, page: {page}""",
        agent=review_loader,
        expected_output='A refined finalized version of the blog post in markdown format'
    )

    content_creation_task = Task(
        description="""Using the insights provided by the research Analyst agent, if have different score , please calculate every score the percentage
        develop an engaging  content, you need use markdown format and translate to chinese. """,
        agent=researcher,
        expected_output=f'A refined finalized version of the blog post in markdown format,need include store: {store} , app_id: {app_id} and country:{country},translate to chinese.'
    )

    # Instantiate a Crew with the review_reader and researcher agents, and the tasks to be performed
    # Set the logging level to 2 (more verbose)
    crew = Crew(
        agents=[review_loader, researcher],
        tasks=[review_loading_task, content_creation_task],
        verbose=2, 
    )

    return app_id,crew

app_id, crew=init_app_crew('Google Play', 'genshin', "us", -1)
crew.kickoff()

2024-05-21 06:53:40,423 [embedchain] [INFO] Swapped std-lib sqlite3 with pysqlite3 for ChromaDb compatibility. Your original version was 3.7.17.
boto3 Bedrock client successfully created!


/data/src/crew-ai-workshop/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


 [DEBUG]: == Working Agent: App Review Research Analyst
 [INFO]: == Starting Task: Use store: Google Play ,app_name:genshin app_id: com.miHoYo.GenshinImpact and country:us, rank: -1, page: 1


> Entering new CrewAgentExecutor chain...
Thought: To get the app reviews for the Genshin Impact app from the Google Play Store, I need to use the get_google_play_app_review tool with the provided app_id, country, rank, and page parameters.

Action: get_google_play_app_review
Action Input: {"app_id": "com.miHoYo.GenshinImpact", "country": "us", "rank": -1, "page": 1}
country='us'
Have continuation token: <google_play_scraper.features.reviews._ContinuationToken object at 0x7f12a343e740>
 

[{'username': 'Meileen Lee', 'content': 'disappointed. primogems are hard to gain even through events..', 'score': 1}, {'username': 'rafiq ansari', 'content': 'Bad this game bad very used data. Many time for start. 1 star', 'score': 1}, {'username': 'Joseph chhangte', 'content': "Play this game u won't regret", 

'# 原神手游评论总结\n\n## 谷歌Play商店, 应用ID: com.miHoYo.GenshinImpact, 国家: 美国\n\n原神是一款获得广泛好评的开放世界冒险游戏。玩家对其精美的画面、生动的场景和有趣的任务赞不绝口。然而,也有一些玩家对游戏提出了一些问题和建议:\n\n### 定价和祈愿系统\n\n一些玩家认为获得原石(游戏内货币)的途径有限,即使是通过活动也很困难。他们还抱怨角色获取的难度很大,需要花费大量金钱或付出极大努力。有玩家呼吁开发商修复这个"不公平的赌博系统"。\n\n> "primogems are hard to gain even through events" - Meileen Lee  \n> "Unbelievable amount of storage taken and the summons are mostly bad and it\'s also hard to get primogems.please fix this unfair gambling system" - patience korankye kesse\n\n### 网络和性能问题  \n\n部分玩家反映游戏在手机上会出现延迟和卡顿的情况,影响了游戏体验。有人还提到游戏占用了大量流量。\n\n> "Bad this game bad very used data. Many time for start." - rafiq ansari\n> "The game is great, but it lags too much on my phone. Whenever I open it and start walking, it keeps lagging." - Zy Ann  \n\n### 游戏内容问题\n\n一些玩家对游戏中的某些内容提出了批评,比如讨厌小精灵派蒙的存在,认为奖励系统不太好,还有人对最新的PVP模式"Windtrace 2024"和某些活动内容表示不满。\n\n> "Annoying ahh paimon that cant be muted. Rewards arent very rewarding. Devs dont listen." - Ash Wholesome\n> "Wind trace 2024 is a 